Now that our two classifiers are ready to predict which stock price will see a 5% increase tomorrow, let's test them!

Something interesting to investigate is if the models can only predict price increase for stocks it has been trained with, or if they can generalize to all stocks as-is, without having to be re-trained on a dataset that would include each new stock's historical data.

In [5]:
!pip install -r requirements.txt

In [2]:
import utils
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from sklearn.preprocessing import MinMaxScaler

In [7]:
# Load RF model
rf_model = pickle.load(open('RF-clf.pickle', 'rb'))

In [8]:
# Load KNN model
kn_model = pickle.load(open('KNN-clf.pickle', 'rb'))

In [9]:
# Load scaler
X_scaler = pickle.load(open('x-scaler.pickle', 'rb'))

Let's build a new test dataset, exclusively made of stocks absent from the initial training and testing sets:

In [10]:
symbol_list = ['HO.PA', 'ALCAR.PA']

X_df = pd.DataFrame()
y_df = pd.DataFrame()

for symbol in symbol_list:
    symbol_X_df = utils.get_stock_feature_dataset(symbol)
    symbol_X_df, symbol_y_df = utils.make_labels_dataset(symbol_X_df)

    # reset index since dates are not required for classification
    X_df = X_df.append(symbol_X_df.reset_index(drop=True), ignore_index=True)
    y_df = y_df.append(symbol_y_df.reset_index(drop=True), ignore_index=True)
    print('Done processing {}! new X_df shape: {}, new y_df shape: {}'.format(symbol, X_df.shape, y_df.shape))

X_df = X_df.astype(float)
X_df.replace(np.inf, np.nan, inplace=True)
X_df.replace(-np.inf, np.nan, inplace=True)
X_df.interpolate(axis=0, limit_direction='both', inplace=True)

print('Check number of NaNs in X_df: {}, and in y_df: {}'.format(X_df.isna().sum().sum(), y_df.isna().sum().sum()))

print('New testing set contains {:.2f}% records labeled as 1'.format(y_df.values.sum()/y_df.shape[0] * 100))

# Scale all values to have the same range:
X_scaled = X_scaler.transform(X_df.values)
y_true = y_df.values.reshape(-1).astype(float)

C:\ProgramData\Miniconda3\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\ProgramData\Miniconda3\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


Done processing HO.PA! new X_df shape: (5279, 86), new y_df shape: (5279, 1)
Done processing ALCAR.PA! new X_df shape: (7341, 86), new y_df shape: (7341, 1)
Check number of NaNs in X_df: 0, and in y_df: 0
New testing set contains 1.62% records labeled as 1


In [17]:
%%time
y_pred = kn_model.predict(X_scaled)

Wall time: 1min 39s


In [20]:
print('Results for KNN model:')
print('\taccuracy: {:.2f}%'.format(accuracy_score(y_true, y_pred) * 100))
print('\tprecision: {:.2f}%'.format(precision_score(y_true, y_pred) * 100))
print('\trecall: {:.2f}%'.format(recall_score(y_true, y_pred) * 100))
print('\tfbeta: {:.3f}'.format(fbeta_score(y_true, y_pred, beta=0.5)))

Results for KNN model:
	accuracy: 98.38%
	precision: 50.00%
	recall: 0.84%
	fbeta: 0.039


In [21]:
y_pred_rf = rf_model.predict(X_scaled)

In [22]:
print('Results for RF model:')
print('\taccuracy: {:.2f}%'.format(accuracy_score(y_true, y_pred_rf) * 100))
print('\tprecision: {:.2f}%'.format(precision_score(y_true, y_pred_rf) * 100))
print('\trecall: {:.2f}%'.format(recall_score(y_true, y_pred_rf) * 100))
print('\tfbeta: {:.3f}'.format(fbeta_score(y_true, y_pred_rf, beta=0.5)))

Results for RF model:
	accuracy: 98.39%
	precision: 100.00%
	recall: 0.84%
	fbeta: 0.041


It appears that both models perform poorly at predicting increase for stocks outside of their training set: indeed, KNN and RF both obtain a recall score close to 0%, and thus a very low fbeta score:
- For the RF model, it means that it is very good at not producing false positives, but very bad at producing true positives
- For the KNN model, it shows even worst performance: not very good at not producing false positives, and very bad at producing true positives!  

Let's now check results obtained on stocks that KNN and RF models have been trained on, and see if their predictions were true. For this, I will make them predict for all of the stocks included in the training set, and check if there was an increase or not on the day after.

I start by building the prediction set:

In [3]:
with open('symbol-list.pickle', 'rb') as f:
    symbol_list = pickle.load(f)

In [37]:
pred_X = pd.DataFrame()

for count, symbol in enumerate(symbol_list):
    # Download data for today, July 15th 2020. This day was not part of the training set.
    X_df = utils.get_stock_feature_dataset(symbol, end=1594836000)
    pred_X = pred_X.append(X_df.tail(1).reset_index(drop=True), ignore_index=True)

    # print progress
    if not count % 5:
        print('Done processing {}! new pred_X shape: {}'.format(symbol, pred_X.shape))

pred_X = pred_X.astype(float)
pred_X.replace(np.inf, np.nan, inplace=True)
pred_X.replace(-np.inf, np.nan, inplace=True)
pred_X.interpolate(axis=0, limit_direction='both', inplace=True)

print('Check number of NaNs in pred_X: {}'.format(X_df.isna().sum().sum()))

# Scale all values to have the same range:
pred_X_scaled = X_scaler.transform(pred_X.values)

Done processing AI.PA! new pred_X shape: (1, 86)
Done processing ETL.PA! new pred_X shape: (6, 86)
Done processing ENGI.PA! new pred_X shape: (11, 86)
Done processing UG.PA! new pred_X shape: (16, 86)
Done processing CRI.PA! new pred_X shape: (21, 86)
Done processing CA.PA! new pred_X shape: (26, 86)
Done processing ORA.PA! new pred_X shape: (31, 86)
Done processing GLE.PA! new pred_X shape: (36, 86)
Done processing TEP.PA! new pred_X shape: (41, 86)
Done processing EUCAR.PA! new pred_X shape: (46, 86)
Done processing AMUN.PA! new pred_X shape: (51, 86)
Done processing CGG.PA! new pred_X shape: (56, 86)
Done processing ALHYG.PA! new pred_X shape: (61, 86)
Done processing ALGBE.PA! new pred_X shape: (66, 86)
Done processing ADP.PA! new pred_X shape: (71, 86)
Done processing AKE.PA! new pred_X shape: (76, 86)
Done processing DGM.PA! new pred_X shape: (81, 86)
Done processing HSB.PA! new pred_X shape: (86, 86)
Done processing NANO.PA! new pred_X shape: (91, 86)
Done processing SBT.PA! new

In [38]:
pred_X.head()

,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,others_dlr,others_cr,cac40_Open,cac40_High,cac40_Low,cac40_Close,sbf120_Open,sbf120_High,sbf120_Low,sbf120_Close
0,136.750000,139.250000,136.250000,137.500000,137.500000,1476815.0,2.283818e+08,257579609.0,0.135315,9.234091e+05,...,1.133677,292.164714,5045.919922,5145.330078,5039.189941,5108.97998,3648.330078,3648.330078,3648.330078,3648.330078
1,91.400002,96.639999,91.139999,95.720001,95.720001,1537193.0,1.913187e+08,229414094.0,-0.041574,1.230640e+06,...,6.451070,108.068857,5045.919922,5145.330078,5039.189941,5108.97998,3648.330078,3648.330078,3648.330078,3648.330078
2,4.540000,4.720000,4.468000,4.668000,4.668000,383016.0,-2.187174e+07,49431257.0,-0.438039,-3.349960e+04,...,3.399020,184.727898,5045.919922,5145.330078,5039.189941,5108.97998,3648.330078,3648.330078,3648.330078,3648.330078
3,2.850000,2.940000,2.810000,2.865000,2.865000,1054505.0,-3.836086e+07,471181671.0,-0.092290,2.008791e+04,...,2.652675,-67.068966,5045.919922,5145.330078,5039.189941,5108.97998,3648.330078,3648.330078,3648.330078,3648.330078
4,27.100000,27.940001,27.100000,27.780001,27.780001,265373.0,-1.837886e+07,25704185.0,-0.525870,-3.173246e+04,...,3.144687,22.378854,5045.919922,5145.330078,5039.189941,5108.97998,3648.330078,3648.330078,3648.330078,3648.330078


In [39]:
# Perform predictions
kn_pred_y = kn_model.predict(pred_X_scaled)
rf_pred_y = rf_model.predict(pred_X_scaled)

In [40]:
# Save KNN preds to a file
outfile = open('KNN-preds.pickle', 'wb')
np.save(outfile, kn_pred_y)
outfile.close()

# Same for RF predictions
outfile = open('RF-preds.pickle', 'wb')
np.save(outfile, rf_pred_y)
outfile.close()

Today is now July 16th 2020, market is closed, let's check yesterday's predictions:

In [54]:
# Construct the true target dataset:

y_true = pd.DataFrame()

for count, symbol in enumerate(symbol_list):
    # Download data for July 15th pre-open and 16th post-close
    X_df = utils.get_stock_feature_dataset(symbol, end=1594922400)
    X_df, y_df = utils.make_labels_dataset(X_df)
    y_true = y_true.append(y_df.tail(1).reset_index(drop=True), ignore_index=True)

    # print progress
    if not count % 5:
        print('Done processing {}! new y_true shape: {}'.format(symbol, y_true.shape))

y_true = y_true.values.reshape(-1).astype(float)

Done processing AI.PA! new y_true shape: (1, 1)
Done processing ETL.PA! new y_true shape: (6, 1)
Done processing ENGI.PA! new y_true shape: (11, 1)
Done processing UG.PA! new y_true shape: (16, 1)
Done processing CRI.PA! new y_true shape: (21, 1)
Done processing CA.PA! new y_true shape: (26, 1)
Done processing ORA.PA! new y_true shape: (31, 1)
Done processing GLE.PA! new y_true shape: (36, 1)
Done processing TEP.PA! new y_true shape: (41, 1)
Done processing EUCAR.PA! new y_true shape: (46, 1)
Done processing AMUN.PA! new y_true shape: (51, 1)
Done processing CGG.PA! new y_true shape: (56, 1)
Done processing ALHYG.PA! new y_true shape: (61, 1)
Done processing ALGBE.PA! new y_true shape: (66, 1)
Done processing ADP.PA! new y_true shape: (71, 1)
Done processing AKE.PA! new y_true shape: (76, 1)
Done processing DGM.PA! new y_true shape: (81, 1)
Done processing HSB.PA! new y_true shape: (86, 1)
Done processing NANO.PA! new y_true shape: (91, 1)
Done processing SBT.PA! new y_true shape: (96,

In [49]:
# Load predictions made yesterday
kn_pred_y = np.load(open('KNN-preds.pickle', 'rb'))
rf_pred_y = np.load(open('RF-preds.pickle', 'rb'))

In [55]:
print('Results for KNN model:')
print('\taccuracy: {:.2f}%'.format(accuracy_score(y_true, kn_pred_y) * 100))
print('\tprecision: {:.2f}%'.format(precision_score(y_true, kn_pred_y) * 100))
print('\trecall: {:.2f}%'.format(recall_score(y_true, kn_pred_y) * 100))
print('\tfbeta: {:.3f}'.format(fbeta_score(y_true, kn_pred_y, beta=0.5)))

Results for KNN model:
	accuracy: 97.92%
	precision: 0.00%
	recall: 0.00%
	fbeta: 0.000


C:\ProgramData\Miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
print('Results for RF model:')
print('\taccuracy: {:.2f}%'.format(accuracy_score(y_true, rf_pred_y) * 100))
print('\tprecision: {:.2f}%'.format(precision_score(y_true, rf_pred_y) * 100))
print('\trecall: {:.2f}%'.format(recall_score(y_true, rf_pred_y) * 100))
print('\tfbeta: {:.3f}'.format(fbeta_score(y_true, rf_pred_y, beta=0.5)))

Results for RF model:
	accuracy: 97.92%
	precision: 0.00%
	recall: 0.00%
	fbeta: 0.000


In [60]:
for sym_idx, sym in enumerate(symbol_list):
    if y_true[sym_idx] == 1:
        print(sym)

ALPHA.PA
DIM.PA
